# IS 362 – Project 3: Chinook Database Query

**Student:** Htet Eaindray Thet  
**Course:** IS 362 – Introduction to Data Analysis  
**Objective:** Join the Chinook tables (Customer, Invoice, InvoiceLine, Track, Album)  
to display each customer's purchased tracks and album titles,  
sorted by Last Name and First Name.


In [6]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')

In [8]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine

DB_PATH = r"C:\Users\htete\Downloads\Chinook_Sqlite.sqlite"


In [9]:

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# list tables
tables = cur.execute(
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
).fetchall()
print("tables in DB:", [t[0] for t in tables])

# show columns for a few important tables
for tbl in ("Customer", "Invoice", "InvoiceLine", "Track", "Album"):
    print("\nColumns in", tbl)
    for col in cur.execute(f"PRAGMA table_info({tbl});").fetchall():
        print(col)  # (cid, name, type, notnull, dflt_value, pk)

conn.close()


tables in DB: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']

Columns in Customer
(0, 'CustomerId', 'INTEGER', 1, None, 1)
(1, 'FirstName', 'NVARCHAR(40)', 1, None, 0)
(2, 'LastName', 'NVARCHAR(20)', 1, None, 0)
(3, 'Company', 'NVARCHAR(80)', 0, None, 0)
(4, 'Address', 'NVARCHAR(70)', 0, None, 0)
(5, 'City', 'NVARCHAR(40)', 0, None, 0)
(6, 'State', 'NVARCHAR(40)', 0, None, 0)
(7, 'Country', 'NVARCHAR(40)', 0, None, 0)
(8, 'PostalCode', 'NVARCHAR(10)', 0, None, 0)
(9, 'Phone', 'NVARCHAR(24)', 0, None, 0)
(10, 'Fax', 'NVARCHAR(24)', 0, None, 0)
(11, 'Email', 'NVARCHAR(60)', 1, None, 0)
(12, 'SupportRepId', 'INTEGER', 0, None, 0)

Columns in Invoice
(0, 'InvoiceId', 'INTEGER', 1, None, 1)
(1, 'CustomerId', 'INTEGER', 1, None, 0)
(2, 'InvoiceDate', 'DATETIME', 1, None, 0)
(3, 'BillingAddress', 'NVARCHAR(70)', 0, None, 0)
(4, 'BillingCity', 'NVARCHAR(40)', 0, None, 0)
(5, 'BillingState', 'NVARCHAR(40)', 0, No

In [14]:
from sqlalchemy import create_engine

DB_PATH = r"C:\Users\htete\Downloads\Chinook_Sqlite.sqlite"
engine = create_engine(f"sqlite:///{DB_PATH}")


In [15]:
sql = """
SELECT
    c.LastName,
    c.FirstName,
    t.Name       AS TrackName,
    al.Title     AS AlbumTitle
FROM Customer c
JOIN Invoice i       ON c.CustomerId = i.CustomerId
JOIN InvoiceLine il  ON i.InvoiceId = il.InvoiceId
JOIN Track t         ON il.TrackId = t.TrackId
JOIN Album al        ON t.AlbumId = al.AlbumId
ORDER BY c.LastName, c.FirstName, t.Name;
"""


In [16]:
import pandas as pd

df = pd.read_sql(sql, engine)
df.head(5)


,LastName,FirstName,TrackName,AlbumTitle
0,Almeida,Roberto,Amor De Muito,Afrociberdelia
1,Almeida,Roberto,Aos Leões,Demorou...
2,Almeida,Roberto,Banditismo Por Uma Questa,Da Lama Ao Caos
3,Almeida,Roberto,Be Good Johnny,The Best Of Men At Work
4,Almeida,Roberto,Burden In My Hand,A-Sides


In [17]:
# Check the shape (number of rows and columns)
df.shape


(2240, 4)

In [18]:
df.head(5)


,LastName,FirstName,TrackName,AlbumTitle
0,Almeida,Roberto,Amor De Muito,Afrociberdelia
1,Almeida,Roberto,Aos Leões,Demorou...
2,Almeida,Roberto,Banditismo Por Uma Questa,Da Lama Ao Caos
3,Almeida,Roberto,Be Good Johnny,The Best Of Men At Work
4,Almeida,Roberto,Burden In My Hand,A-Sides


In [19]:
print(f"Unique customers: {df['LastName'].nunique()}")
print(f"Unique tracks: {df['TrackName'].nunique()}")
print(f"Total records: {len(df)}")


Unique customers: 59
Unique tracks: 1888
Total records: 2240


In [20]:
df.to_csv("Chinook_Customer_Tracks.csv", index=False)


### IS 362 – Project 3

In this project, I used Python (pandas and SQLAlchemy) to query the **Chinook** database.
The goal was to join the following tables:

- Customer  
- Invoice  
- InvoiceLine  
- Track  
- Album  

The query returns each customer's purchased tracks along with the album title,
sorted by **LastName** and **FirstName**.

**Output summary:**  
- Total records: 2,240  
- Columns: LastName, FirstName, TrackName, AlbumTitle  
